In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import missingno as msno
import matplotlib.pyplot as plt
from datetime import date
today = date.today()
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import sklearn
from wordcloud import WordCloud
import string
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
import pyLDAvis
import pyLDAvis.sklearn
import gensim
from itertools import combinations
from sklearn.metrics import accuracy_score, f1_score, hamming_loss,jaccard_score
from sklearn.tree import DecisionTreeClassifier

C:\Users\jerom\anaconda3\envs\PROJET5\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [ ]:
#Lecture du Fichier
#df = pd.read_csv('posts_after_prepocessing.csv', sep = ',', encoding='UTF-8')

In [4]:
import pickle
with open('df_afterPreprocessing.pickle', 'rb') as file:
    df = pickle.load(file)

In [5]:
#Fonction de preprocessing
stop_words = set(stopwords.words('English'))
def removeStopWord(Word_list):
    filtered_Word_list = Word_list[:] #make a copy of the Word_list
    for Word in Word_list: # iterate over Word_list
        if Word.lower() in stop_words: 
            filtered_Word_list.remove(Word) # remove Word from filtered_Word_list if it is a stopword
    return filtered_Word_list

# Instantiate stemmers
porter = PorterStemmer()
#Fonction de lemmatisation
def lemmatisation(Word_list):
    Words = Word_list[:] #make a copy of the Word_list
    Words = [porter.stem(word) for word in Words]
    return Words

#Fonction qui supprime le tag si celui ci n'appartient au TOP        
def removeNotTop100(Word_list):
    filtered_Word_list = Word_list[:] #make a copy of the Word_list
    for Word in Word_list: # iterate over Word_list
        if Word not in Top100: 
            filtered_Word_list.remove(Word) # remove Word from filtered_Word_list if it is a stopword
    if len(filtered_Word_list) == 0:
        return None
    else:
        return filtered_Word_list
    
#Fonction qui supprime le mot si seulement du numérique        
def removeOnlyNumeric(Word_list):
    word_list = Word_list[:] #make a copy of the Word_list
    for Word in Word_list: # iterate over Word_list
        if Word.isnumeric(): 
            word_list.remove(Word) # remove Word from filtered_Word_list if it is a stopword
    return word_list
    
#Fonction qui traite le C#
def processCSharp(Word_list):
    word_list = Word_list[:] #make a copy of the Word_list
    for index, value in enumerate(word_list):
        if value == '#':
            word_list.remove(value) # remove Word from filtered_Word_list if it is a stopword
            word_list[index-1] = 'c#'
    return word_list

# Fonction qui ne fait rien pour exploiter le pré traitement que nous avons réalisé
def dummy(doc):
    return doc

#Fonction qui renvoi différents scores afin d'évaluer les modèles
def print_score(y_test, y_pred):
    print("Hamming loss : {}".format(hamming_loss(y_test, y_pred)))
    print('Subset Accuracy : ', accuracy_score(y_test, y_pred, normalize=True, sample_weight=None))
    print('F1-score : ', f1_score(y_test, y_pred, average='micro'))
    print('Jaccard : ', jaccard_score(y_test, y_pred, average='micro'))

In [6]:
#Préprocessing du champ Tittle et Body
df['TitleBody'] = df['TitleBody'].map(lambda x: BeautifulSoup(x, "html.parser").get_text())
df['TitleBody'] = df['TitleBody'].map(lambda x: x.replace('\n', ' '))
df['TitleBody'] = df['TitleBody'].map(lambda x: x.replace(':', ''))

punct = string.punctuation
for c in punct:
    if c != '#':
        df['TitleBody'] = df['TitleBody'].map(lambda x: x.replace(c, ''))

#Création des nuages de mots avec NLTK
df['TitleBody_final'] = df.apply(lambda row: nltk.word_tokenize(row['TitleBody'],language='english'), axis=1)

#Supression des StopWord
df['TitleBody_final'] = df.apply(lambda row:removeStopWord(row['TitleBody_final']), axis=1)

#Lemmatisation
#df['TitleBody_final'] = df.apply(lambda row:lemmatisation(row['TitleBody_final']), axis=1)

#Traitement du C#
df['TitleBody_final'] = df.apply(lambda row:processCSharp(row['TitleBody_final']), axis=1)

#Suppressin des numériques
df['TitleBody_final'] = df.apply(lambda row:removeOnlyNumeric(row['TitleBody_final']), axis=1)

In [ ]:
df.head(5)

In [7]:
#Utilisation de la librairie MultilabelBinarize pour créer un matrice de Tags
from sklearn.preprocessing import MultiLabelBinarizer

tags = df["processed_tags_final"]

mlb = MultiLabelBinarizer()
tags_mlb = mlb.fit_transform(df["processed_tags_final"].to_numpy())

## Création du jeu d'entrainement et du jeu de test 

In [8]:
from sklearn.model_selection import train_test_split

X = df['TitleBody_final']
y = tags_mlb

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=.25)

## Vectorisation avec TF-IDF

In [9]:
vectorizer = TfidfVectorizer(tokenizer=dummy,preprocessor=dummy,
                            max_features=12000)
vectorizer.fit(X_train)
X_train_vec = vectorizer.transform(X_train)
X_test_vec = vectorizer.transform(X_test)

C:\Users\jerom\anaconda3\envs\PROJET5\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


## Modèle Naive Bayse après TF_IDF
* Utilisation de la méthode OneVsRest qui calcule toutes les étiquettes de sortie (Attention aux performances car n modèles suivant les tags de sortie)
* J'utilise les données après un TF_IDF

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB

nb_clf = OneVsRestClassifier(MultinomialNB(fit_prior=True, 
                                           class_prior=None))
nb_clf.fit(X_train_vec, y_train)

In [ ]:
y_pred_nb = nb_clf.predict(X_test_vec)
print_score(y_test, y_pred_nb)

## Modèle Arbre de décision après NMF (Comme réduction de dimension)

In [ ]:
#D'abord réalisation d'une NMF avec le paramètre optimum calculté précédemment (8 Topics)
from sklearn.decomposition import NMF
k = 8
# create the model, specifiying the initialization strategy and the number of topics to produce
model = NMF(n_components=k,init="nndsvd",random_state=42,max_iter = 300) 
# apply the model and extract the two factor matrices
W_train = model.fit_transform(X_train_vec)
W_test = model.fit_transform(X_test_vec)

In [ ]:
#Arbre de décision
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(W_train, y_train)

In [ ]:
y_pred_nb = clf.predict(W_test)
print_score(y_test, y_pred_nb)

## Modèle Arbre de décision sans NMF

In [ ]:
#Arbre de décision
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train_vec, y_train)

In [ ]:
y_pred_nb = clf.predict(X_test_vec)
print_score(y_test, y_pred_nb)

## Modèle Arbre de décision après TruncateSVD
* Contrairement à l'ACP TruncateSVD supporte les matrices creuses

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=1000, n_iter=7, random_state=42)
svd.fit(X_train_vec)
X_train_SVD = svd.transform(X_train_vec)
X_test_SVD = svd.transform(X_test_vec)

#Arbre de décision
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train_SVD, y_train)

In [ ]:
y_pred_nb = clf.predict(X_test_SVD)
print_score(y_test, y_pred_nb)

In [ ]:
X_train_vec.shape

## Modèle Arbre de décision après ACP

In [10]:
from sklearn import decomposition
# choix du nombre de composantes à calculer
n_comp = 50

features = df.columns

# Calcul des composantes principales
pca = decomposition.PCA(n_components=n_comp,random_state=42)
pca.fit(X_train_vec.toarray())
X_train_PCA = pca.transform(X_train_vec.toarray())
X_test_PCA = pca.transform(X_test_vec.toarray())

#Arbre de décision
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train_PCA, y_train)

DecisionTreeClassifier(random_state=42)

In [11]:
y_pred_nb = clf.predict(X_test_PCA)
print_score(y_test, y_pred_nb)

Hamming loss : 0.021903506459267072
Subset Accuracy :  0.15317690482467702
F1-score :  0.3414711477488903
Jaccard :  0.2058879755304913


In [ ]:
mlb.classes_

In [21]:
#Fonction qui compte les bonnes prédictions et les mauvaises prédictions par tags
def scoreTags():
    resultat = []
    for index, word in enumerate(mlb.classes_):
        GoodPrediction = 0
        BadPrediction = 0
        for i in range(0,len(y_test)):
            if (y_test[i][index] == 1):
                if (y_test[i][index] == y_pred_nb[i][index]):
                    GoodPrediction += 1
                else:
                    BadPrediction +=1
        resultat.append((word,GoodPrediction,BadPrediction))
    return resultat

In [22]:
comptagePrediction = scoreTags()

In [ ]:
for res in y_test:

In [ ]:
len(y_test)

In [23]:
comptagePrediction

[('.net', 12, 121),
 ('ajax', 3, 20),
 ('algorithm', 2, 29),
 ('android', 120, 114),
 ('android-studio', 10, 8),
 ('angular', 8, 38),
 ('angularjs', 1, 34),
 ('arrays', 43, 54),
 ('asp.net', 3, 26),
 ('asp.net-mvc', 2, 31),
 ('bash', 46, 75),
 ('branch', 5, 22),
 ('c', 10, 49),
 ('c#', 78, 190),
 ('c++', 33, 96),
 ('c++11', 1, 17),
 ('class', 0, 19),
 ('cocoa-touch', 0, 20),
 ('collections', 1, 23),
 ('command-line', 7, 33),
 ('css', 102, 66),
 ('database', 1, 30),
 ('dataframe', 7, 26),
 ('date', 12, 30),
 ('datetime', 8, 29),
 ('debugging', 0, 20),
 ('dictionary', 7, 30),
 ('django', 0, 23),
 ('docker', 19, 13),
 ('dom', 3, 17),
 ('eclipse', 5, 26),
 ('exception', 2, 25),
 ('file', 3, 26),
 ('forms', 1, 16),
 ('function', 2, 26),
 ('generics', 1, 15),
 ('git', 234, 25),
 ('github', 12, 44),
 ('go', 0, 17),
 ('google-chrome', 1, 24),
 ('html', 71, 135),
 ('http', 2, 32),
 ('intellij-idea', 1, 17),
 ('ios', 21, 79),
 ('iphone', 2, 23),
 ('java', 178, 178),
 ('java-8', 0, 15),
 ('javasc